In [ ]:
import pandas as pd

In [ ]:
VERBOSE = True
FP_VCF = '/Users/Kwat/binf/guardiome/g/genomics/data/tester/0.05/variant/0.05.concat.concat.rename_chr_sort.ann_snpeff.ann_clinvar.rename_chr_sort.vcf'

In [ ]:
# Read VCF
def read_vcf(fp_vcf, verbose=False):
    meta_information = []
    header_line = None
    header = None
    vcf_data = None

    with open(fp_vcf) as f:
        for i, line in enumerate(f):
            if line.startswith('##'):
                meta_information.append(line.strip())
            elif line.startswith('#CHROM'):
                header_line = i
                print('header starts at line {}\n'.format(header_line))
                header = line.strip()
            else:
                break
    vcf_data = pd.read_csv(FP_VCF, sep='\t', skiprows=header_line)

    if verbose:
        print('Meta-information\n{}\n'.format(meta_information))
        print('header\n{}\n'.format(header))
        print('VCF\n{}\n'.format(vcf_data.head()))
    return meta_information, header_line, header, vcf_data

meta_information, header_line, header, vcf_data = read_vcf(FP_VCF, verbose=True)

In [ ]:
# Add FORMAT and SAMPLE
def add_format_sample(vcf_data, sample_id, verbose=False):
    meta_information.append('##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">')
    meta_information.append('##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth; some reads may have been filtered">')
    vcf_data['FORMAT'] = 'GT:DP'
    vcf_data[SAMPLE_ID] = '1/1:999'
    vcf_data['QUAL'] = 999
    vcf_data['FILTER'] = 'PASS'

    if VERBOSE:
        print('VCF\n{}\n'.format(vcf_data.head()))

SAMPLE_ID = '738_0.05'
add_format_sample(SAMPLE_ID, verbose=VERBOSE)

In [ ]:
with open('/Users/Kwat/Downloads/snps_to_add1.txt', 'r') as f:
    for line in f.readlines()[1:]:
        rs, chrom, start, stop, ref, alt = [e.strip() for e in line.split('\t')]
        row = {'#CHROM':chrom,
               'POS':start,
               'ID':rs,
               'REF':ref,
               'ALT':alt,
               'QUAL':999,
               'FILTER':'PASS',
               'INFO':'',
               'FORMAT':'GT:DP',
               '738_0.05':'1/1:999'}
        vcf_data = vcf_data.append(row, ignore_index=True)
if VERBOSE:
    print('VCF\n{}\n'.format(vcf_data.tail()))

In [ ]:
def write_vcf(fp_vcf, suffix='editted'):
    assert fp_vcf.endswith('.vcf')
    # Make output filename
    sep_idx = fp_vcf.find('.vcf')
    prefix, vcf_suffix = fp_vcf[:sep_idx], fp_vcf[sep_idx:]
    fp_output = prefix + '.{}'.format(suffix) + vcf_suffix

    # Save editted VCF
    with open(fp_output, 'w') as f:
        f.write('\n'.join(meta_information))
        f.write('\n')
        vcf_data.to_csv(f, sep='\t', index=False)
    print('Written: {}'.format(fp_output))

write_vcf('/Users/Kwat/Desktop/0.05.concat.concat.rename_chr_sort.ann_snpeff.ann_clinvar.rename_chr_sort.vcf', suffix='add_fake_var')

In [ ]:
import os
import sys
CODE_BASEPATH = os.path.join(os.environ['GUARDIOME'], 'helixa', 'code')
sys.path.insert(0, CODE_BASEPATH)
from genomics import genomic_tools

In [ ]:
vcf_dir = '/Users/Kwat/Desktop'
G_VCF = os.path.join(vcf_dir, '0.05.concat.concat.rename_chr_sort.ann_snpeff.ann_clinvar.rename_chr_sortadd_fake_var.vcf.gz')
G_VCF_INDEX = os.path.join(vcf_dir, 'rs_id_index.pickle.gz')
G_VCF_TABLE = os.path.join(vcf_dir, 'variant.hdf5')

print('G_VCF\t{}'.format(G_VCF))
print('G_VCF_INDEX\t{}'.format(G_VCF_INDEX))
print('G_VCF_TABLE\t{}'.format(G_VCF_TABLE))
di = genomic_tools.DataInterface(vcf_filename=G_VCF,
                                 variant_rs_id_index_filename=G_VCF_INDEX,
                                 variant_table_filename=G_VCF_TABLE)